# Goal
The goal of this notebook is to compare the performance of multi-objective probability of improvement backed by a multi-
objective homogeneous random forest to random search.

# Methodology
We only need to allow each strategy to execute for some fixed number of iterations. We can subsequently process the resulting observations to estimate the size of the pareto frontier over time. 

Specifically, let's first run both strategies on a 2D hypersphere, 10 times each and compare their average performance.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from mlos.OptimizerEvaluationTools.ObjectiveFunctionConfigStore import objective_function_config_store
from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory
from mlos.OptimizerEvaluationTools.SyntheticFunctions.Hypersphere import Hypersphere

from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Optimizers.ParetoFrontier import ParetoFrontier

from mlos.Spaces import Point

objective_function_config = objective_function_config_store.get_config_by_name("multi_objective_2_mutually_exclusive_polynomials")
objective_function_config.multi_objective_nested_polynomial_config.num_objectives = 4
objective_function_config

{
  "implementation": "MultiObjectiveNestedPolynomialObjective",
  "multi_objective_nested_polynomial_config.num_objectives": 4,
  "multi_objective_nested_polynomial_config.objective_function_implementation": "NestedPolynomialObjective",
  "multi_objective_nested_polynomial_config.nested_polynomial_objective_config.num_nested_polynomials": 2,
  "multi_objective_nested_polynomial_config.nested_polynomial_objective_config.nested_function_implementation": "PolynomialObjective",
  "multi_objective_nested_polynomial_config.nested_polynomial_objective_config.polynomial_objective_config.seed": 17,
  "multi_objective_nested_polynomial_config.nested_polynomial_objective_config.polynomial_objective_config.input_domain_dimension": 2,
  "multi_objective_nested_polynomial_config.nested_polynomial_objective_config.polynomial_objective_config.input_domain_min": -1024,
  "multi_objective_nested_polynomial_config.nested_polynomial_objective_config.polynomial_objective_config.input_domain_width": 2048,


In [3]:
objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)

In [4]:
objective_function.parameter_space

  Name: domain
  Dimensions:
    polynomial_id: {0, 1}

  IF polynomial_id IN {1} THEN (
    Name: domain_1
    Dimensions:
      x_0: [-1024.00, 1024.00]
      x_1: [-1024.00, 1024.00]
  )

  IF polynomial_id IN {0} THEN (
    Name: domain_0
    Dimensions:
      x_0: [-1024.00, 1024.00]
      x_1: [-1024.00, 1024.00]
  )

In [5]:
objective_function.output_space

  Name: output_space
  Dimensions:
    y0: [-inf, inf]
    y1: [-inf, inf]
    y2: [-inf, inf]
    y3: [-inf, inf]

In [6]:
optimization_problem = objective_function.default_optimization_problem

optimizer_config = bayesian_optimizer_config_store.get_config_by_name("default_multi_objective_optimizer_config")
optimizer_config.experiment_designer_config.random_search_optimizer_config.num_samples_per_iteration = 10000
optimizer_config.experiment_designer_config.fraction_random_suggestions = 0.3
optimizer_config.homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.n_new_samples_before_refit = 1
print(optimizer_config)

{
  "surrogate_model_implementation": "MultiObjectiveHomogeneousRandomForest",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 10,
  "homogeneous_random_forest_regression_model_config.n_estimators": 10,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 0.7,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.7,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.max_depth": 0,
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.min_samples_s

In [7]:
import concurrent.futures
from run_optimization import run_optimization

num_runs = 10
num_iterations_per_run = 200
max_workers = 10
optimizers = []

with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
    outstanding_futures = set()
    for run_id in range(num_runs):
        future = executor.submit(run_optimization, optimization_problem, optimizer_config, objective_function, run_id, num_iterations_per_run)
        outstanding_futures.add(future)
        
    done_futures, outstanding_futures = concurrent.futures.wait(outstanding_futures, return_when=concurrent.futures.ALL_COMPLETED)
    print(f"Completed {len(done_futures)} futures")
    for future in done_futures:
        optimizer = future.result()
        optimizers.append(optimizer)

Completed 10 futures


In [8]:
objectives_dfs_by_run = []
for optimizer in optimizers:
    _, objectives_df, _ = optimizer.get_all_observations()
    objectives_dfs_by_run.append(objectives_df)

In [9]:
pareto_volumes_over_time_per_run = [[] for i in range(10, num_iterations_per_run, 10)]
for run_id, objectives_df in enumerate(objectives_dfs_by_run):
    for i, j in enumerate(range(10, num_iterations_per_run, 10)):
        pareto_frontier = ParetoFrontier(optimization_problem=optimization_problem, objectives_df=objectives_df[:j])
        pareto_volume_estimator = pareto_frontier.approximate_pareto_volume(num_samples=1000000)
        lower_bound, upper_bound = pareto_volume_estimator.get_two_sided_confidence_interval_on_pareto_volume(alpha=0.05)
        pareto_volume = (lower_bound + upper_bound) / 2
        pareto_volumes_over_time_per_run[i].append(pareto_volume)


In [10]:
objectives_dfs_by_run

[               y0            y1            y2            y3
 0   -3.014268e+06 -6.507907e+06 -1.914695e+06 -6.773102e+06
 1   -1.348892e+07 -1.450022e+07 -1.199847e+07 -1.611669e+07
 2   -7.729165e+06 -1.083064e+07 -6.737699e+06 -9.923122e+06
 3    1.424216e+06 -3.978612e+06 -1.098041e+06 -7.144029e+06
 4   -4.512520e+06 -1.424680e+06  1.821611e+06 -8.069914e+06
 ..            ...           ...           ...           ...
 195 -1.098632e+07 -1.286445e+07 -9.586594e+06 -1.307947e+07
 196 -1.087003e+07 -1.263525e+07 -9.495581e+06 -1.292930e+07
 197 -1.076186e+07 -1.255144e+07 -9.396178e+06 -1.280555e+07
 198 -1.143540e+07 -1.293228e+07 -1.003957e+07 -1.358158e+07
 199 -4.407753e+06 -6.425547e+06 -3.860822e+06 -5.814113e+06
 
 [200 rows x 4 columns],
                y0            y1            y2            y3
 0   -3.014268e+06 -6.507907e+06 -1.914695e+06 -6.773102e+06
 1   -1.348892e+07 -1.450022e+07 -1.199847e+07 -1.611669e+07
 2   -7.729165e+06 -1.083064e+07 -6.737699e+06 -9.923122e+

In [11]:
average_pareto_volume_by_iteration = [sum(pareto_volumes) / len(pareto_volumes) for pareto_volumes in pareto_volumes_over_time_per_run]

In [12]:
pareto_volumes_per_run_over_time = [[0 for i in range(10, num_iterations_per_run, 10)] for objectives_df in objectives_dfs_by_run]
for run_id, objectives_df in enumerate(objectives_dfs_by_run):
    for col, iteration_number in enumerate(range(10, num_iterations_per_run, 10)):
        pareto_volumes_per_run_over_time[run_id][col] = pareto_volumes_over_time_per_run[col][run_id]

In [13]:
average_pareto_volume_by_iteration

[3.782277545826766e+28,
 4.243461224512427e+28,
 5.554352292050646e+28,
 7.296516207827884e+28,
 7.505288137486274e+28,
 7.588378949135266e+28,
 7.761055564577536e+28,
 8.114979162953544e+28,
 8.2257700424737e+28,
 8.34045425567324e+28,
 8.4012442186738e+28,
 8.439838749396881e+28,
 8.573911774012036e+28,
 8.678028786187654e+28,
 8.68904140309881e+28,
 8.802025469717717e+28,
 8.819323017759922e+28,
 8.819320299057006e+28,
 8.867822113542577e+28]

In [14]:
# Now let's do the random ones :)
#
num_runs = 10
objectives_dfs_by_random_run = []

for run_id in range(num_runs):
    suggestions_df = objective_function.parameter_space.random_dataframe(num_iterations_per_run)
    objectives_df = objective_function.evaluate_dataframe(suggestions_df)
    objectives_dfs_by_random_run.append(objectives_df)
    
pareto_volumes_over_time_per_random_run = [[] for i in range(10, num_iterations_per_run, 10)]
for run_id, objectives_df in enumerate(objectives_dfs_by_random_run):
    for i, j in enumerate(range(10, num_iterations_per_run, 10)):
        pareto_frontier = ParetoFrontier(optimization_problem=optimization_problem, objectives_df=objectives_df[:j])
        pareto_volume_estimator = pareto_frontier.approximate_pareto_volume(num_samples=1000000)
        lower_bound, upper_bound = pareto_volume_estimator.get_two_sided_confidence_interval_on_pareto_volume(alpha=0.05)
        pareto_volume = (lower_bound + upper_bound) / 2
        pareto_volumes_over_time_per_random_run[i].append(pareto_volume)

average_random_pareto_volume_by_iteration = [sum(pareto_volumes) / len(pareto_volumes) for pareto_volumes in pareto_volumes_over_time_per_random_run]

In [15]:
pareto_volumes_per_random_run_over_time = [[0 for i in range(10, num_iterations_per_run, 10)] for run_id in range(num_runs)]
for run_id in range(num_runs):
    for col, iteration in enumerate(range(10, num_iterations_per_run, 10)):
        pareto_volumes_per_random_run_over_time[run_id][col] = pareto_volumes_over_time_per_random_run[col][run_id]

In [16]:
average_random_pareto_volume_by_iteration

[4.638207003197431e+27,
 8.421370727843991e+27,
 1.4653899105679262e+28,
 1.8532396431137674e+28,
 2.534120909154592e+28,
 2.925359915889117e+28,
 3.0171270740893833e+28,
 3.870654443943665e+28,
 4.499221884435526e+28,
 4.499351067725514e+28,
 4.983093693324612e+28,
 5.1110221416228124e+28,
 5.39298569257736e+28,
 5.542945900771974e+28,
 5.589534775088396e+28,
 5.589956269722926e+28,
 5.590307144183719e+28,
 5.5921765165125105e+28,
 5.592015434303628e+28]

In [27]:
import plotly.graph_objs as go
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=[i for i in range(10, num_iterations_per_run, 10)],
    y=average_pareto_volume_by_iteration,
    name='Guided Optimization: Average Pareto Volume vs. Iteration'
))

fig.add_trace(go.Scatter(
    x=[i for i in range(10, num_iterations_per_run, 10)],
    y=average_random_pareto_volume_by_iteration,
    name='Random Search: Average Pareto Volume vs. Iteration'
))

for run_id, guided_pareto_volume in enumerate(pareto_volumes_per_run_over_time):
    fig.add_trace(go.Scatter(
    x=[i for i in range(10, num_iterations_per_run, 10)],
    y=guided_pareto_volume,
    name=f'Guided Run {run_id}',
    marker={"color":"LightSkyBlue"}
))

for run_id, guided_pareto_volume in enumerate(pareto_volumes_per_random_run_over_time):
    fig.add_trace(go.Scatter(
    x=[i for i in range(10, num_iterations_per_run, 10)],
    y=guided_pareto_volume,
    name=f'Random Run {run_id}',
    marker={"color":"IndianRed"}
))
    
fig.show()

In [18]:
import pandas as pd
from IPython.display import display
params_df = objective_function.parameter_space.random_dataframe(20)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(params_df)

,polynomial_id,domain_1.x_0,domain_1.x_1,domain_0.x_0,domain_0.x_1
0,1,96.477118,-328.781789,NaN,NaN
1,1,-224.662940,-949.081050,NaN,NaN
2,0,NaN,NaN,154.794425,-196.518613
3,0,NaN,NaN,-919.034475,-869.445477
4,0,NaN,NaN,-738.772247,809.065332
5,0,NaN,NaN,360.898655,-415.077689
6,0,NaN,NaN,-182.827015,102.530495
7,0,NaN,NaN,-931.113814,-711.309303
8,0,NaN,NaN,-451.121560,-949.100606
9,1,-998.885189,-392.468821,NaN,NaN


In [19]:
objectives_df = objective_function.evaluate_dataframe(params_df)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(objectives_df)

,y0,y1,y2,y3
0,-1.253015e+05,-2.217479e+05,-3.922192e+05,-9.566909e+05
1,-5.594802e+06,-4.852158e+06,-6.022213e+06,-9.038416e+06
2,-2.554263e+05,-6.506797e+04,1.012965e+05,-4.391467e+05
3,-1.137471e+07,-1.391799e+07,-7.971767e+06,-1.448815e+07
4,-5.296291e+06,-7.314693e+05,1.944234e+06,-8.340779e+06
5,-1.301130e+06,-2.249941e+05,4.924440e+05,-2.108323e+06
6,-2.692540e+05,1.011498e+04,3.731838e+04,-3.025241e+05
7,-1.043118e+07,-1.107544e+07,-6.829383e+06,-1.228052e+07
8,-5.621647e+06,-1.033573e+07,-4.312472e+06,-9.985045e+06
9,-7.023880e+06,-1.115852e+07,-6.224329e+06,-9.898979e+06


In [20]:
multi_objective_gof = optimizers[2].compute_surrogate_model_goodness_of_fit()
for objective_name, gof in multi_objective_gof:
    print(objective_name)
    print(gof.to_json(indent=2))

y0
{
  "last_refit_iteration_number": 183,
  "observation_count": 200,
  "prediction_count": 200,
  "data_set_type": 0,
  "mean_absolute_error": 523162.982437192,
  "root_mean_squared_error": 930689.0897782453,
  "relative_absolute_error": 0.10726915752534819,
  "relative_squared_error": 0.1644068685974282,
  "coefficient_of_determination": 0.972970381557988,
  "prediction_90_ci_hit_rate": 0.945,
  "sample_90_ci_hit_rate": 0.96
}
y1
{
  "last_refit_iteration_number": 183,
  "observation_count": 200,
  "prediction_count": 200,
  "data_set_type": 0,
  "mean_absolute_error": 582433.7917579361,
  "root_mean_squared_error": 1317834.138554726,
  "relative_absolute_error": 0.10669155292054264,
  "relative_squared_error": 0.21169114520950996,
  "coefficient_of_determination": 0.9551868590398862,
  "prediction_90_ci_hit_rate": 0.925,
  "sample_90_ci_hit_rate": 0.95
}
y2
{
  "last_refit_iteration_number": 183,
  "observation_count": 200,
  "prediction_count": 200,
  "data_set_type": 0,
  "mean_a

In [21]:
from mlos.Optimizers.RegressionModels.GoodnessOfFitMetrics import DataSetType

params_df = objective_function.parameter_space.random_dataframe(1000)
objectives_df = objective_function.evaluate_dataframe(params_df)
features_df = optimization_problem.construct_feature_dataframe(params_df)

optimizer = optimizers[1]
multi_objective_gof = optimizer.surrogate_model.compute_goodness_of_fit(
    features_df=features_df,
    targets_df=objectives_df,
    data_set_type=DataSetType.TEST
)

for objective_name, gof in multi_objective_gof:
    print(objective_name)
    print(gof.to_json(indent=2))

y0
{
  "last_refit_iteration_number": 191,
  "observation_count": 1000,
  "prediction_count": 1000,
  "data_set_type": 2,
  "mean_absolute_error": 1580468.47795201,
  "root_mean_squared_error": 2241537.6211048462,
  "relative_absolute_error": 0.5802911875281607,
  "relative_squared_error": 0.6665919870261066,
  "coefficient_of_determination": 0.555655122832587,
  "prediction_90_ci_hit_rate": 0.741,
  "sample_90_ci_hit_rate": 0.831
}
y1
{
  "last_refit_iteration_number": 191,
  "observation_count": 1000,
  "prediction_count": 1000,
  "data_set_type": 2,
  "mean_absolute_error": 2400285.097398636,
  "root_mean_squared_error": 3669763.586744493,
  "relative_absolute_error": 0.8497758627314612,
  "relative_squared_error": 1.0167720082249887,
  "coefficient_of_determination": -0.03382531670987632,
  "prediction_90_ci_hit_rate": 0.572,
  "sample_90_ci_hit_rate": 0.718
}
y2
{
  "last_refit_iteration_number": 191,
  "observation_count": 1000,
  "prediction_count": 1000,
  "data_set_type": 2,
 

In [22]:
for optimizer in optimizers:
    params_df, objectives_df, _ = optimizer.get_all_observations()
    display(optimizer.pareto_frontier.pareto_df)
    display(params_df.loc[optimizer.pareto_frontier.pareto_df.index])
    

,y0,y1,y2,y3
60,-1.663000e+07,-1.878032e+07,-1.460717e+07,-1.975343e+07


,polynomial_id,domain_0.x_0,domain_0.x_1,domain_1.x_0,domain_1.x_1
60,1,NaN,NaN,-1023.52,-1009.79


,y0,y1,y2,y3
142,-1.667132e+07,-1.878047e+07,-1.465112e+07,-1.980315e+07


,polynomial_id,domain_0.x_0,domain_0.x_1,domain_1.x_0,domain_1.x_1
142,1,NaN,NaN,-1020.69,-1014.6


,y0,y1,y2,y3
192,-1.667106e+07,-1.881531e+07,-1.464511e+07,-1.980233e+07
129,-1.665279e+07,-1.865497e+07,-1.465270e+07,-1.978449e+07


...

,polynomial_id,domain_0.x_0,domain_0.x_1,domain_1.x_0,domain_1.x_1
96,1,NaN,NaN,-1023.61,-1015.24


,y0,y1,y2,y3
155,-1.662696e+07,-1.878287e+07,-1.460354e+07,-1.974977e+07


,polynomial_id,domain_0.x_0,domain_0.x_1,domain_1.x_0,domain_1.x_1
155,1,NaN,NaN,-1023.95,-1009.24


,y0,y1,y2,y3
126,-1.680704e+07,-1.891004e+07,-1.477435e+07,-1.996493e+07


,polynomial_id,domain_0.x_0,domain_0.x_1,domain_1.x_0,domain_1.x_1
126,1,NaN,NaN,-1022.79,-1020.51


In [23]:
objective_function.parameter_space

  Name: domain
  Dimensions:
    polynomial_id: {0, 1}

  IF polynomial_id IN {1} THEN (
    Name: domain_1
    Dimensions:
      x_0: [-1024.00, 1024.00]
      x_1: [-1024.00, 1024.00]
  )

  IF polynomial_id IN {0} THEN (
    Name: domain_0
    Dimensions:
      x_0: [-1024.00, 1024.00]
      x_1: [-1024.00, 1024.00]
  )

In [24]:
suggestion = objective_function.parameter_space.random()
print(suggestion)

{
  "polynomial_id": 0,
  "domain_0.x_0": -994.4147196191955,
  "domain_0.x_1": -904.7798423086522
}


In [25]:
suggestion.domain_1.x_0 = -1024
suggestion.domain_1.x_1 = -1024

AttributeError: This Point does not have a domain_1 attribute.

In [ ]:
print(suggestion)

In [ ]:
value = objective_function.evaluate_point(suggestion)
print(value)

In [ ]:

features_df = optimization_problem.construct_feature_dataframe(suggestion.to_dataframe())
mo_prediction = optimizer.surrogate_model.predict(features_df)

In [ ]:
for name, pred in mo_prediction:
    print(pred)

In [ ]:
import os
os.getpid()

In [ ]:
params_df = objective_function.parameter_space.random_dataframe(10)
features_df = optimization_problem.construct_feature_dataframe(params_df)
optimizer.experiment_designer.utility_function(features_df).describe()